# Chapter 4-A Top-Down Parsing

The syntax analyzer performs the major syntax checking (the inherently recursive part) of the source program.

#### Example

$L = \{a^nb^n \mid n \ge 1\}$, i.e., the language of all nested parentheses (or begin-ends), is not regular but is context-free.

For example, `()`, `(())`, `begin begin end end`.

The syntax of a programming language is defined by a context-free grammar but there are language features that cannot be described by a context-free grammar. They must be handled by the semantic analyzer.

#### Example

$L = \{wzw \mid w, z \in \{a, b\}^*\}$, i.e., the language abstracting the feature that identifiers are declared before their use, is not context-free but is context-sensitive (semantic analyzer).

#### Example

$
\begin{eqnarray}
E &\rightarrow& E + E \\
&\mid& E * E \\
&\mid& [id] \\
&\mid& [const]
\end{eqnarray}
$

<h3><center><i>Example Parse Tree</i></center></h3>

<img src="./res/04/4_1.png" width="300px" alt="Example Parse Tree"/>

A leftmost derivation:

$
\begin{eqnarray}
E &\Rightarrow_{lm}& E * E \\
&\Rightarrow_{lm}& [id] * E \\
&\Rightarrow_{lm}& [id] * E + E \\
&\Rightarrow_{lm}& [id] * [id] + E \\
&\Rightarrow_{lm}& [id] * [id] + [id]
\end{eqnarray}
$

A rightmost derivation:

$
\begin{eqnarray}
E &\Rightarrow_{rm}& E * E & \\
&\Rightarrow_{rm}& E * E + E \\
&\Rightarrow_{rm}& E * E + [id] \\
&\Rightarrow_{rm}& E * [id] + [id] \\
&\Rightarrow_{rm}& [id] * [id] + [id]
\end{eqnarray}
$

#### Note
There is a one-to-one correspondance between a parse tree and a right-most derivation.

#### Note
There is a one-to-one correspondance between a parse tree and a left-most derivation.

## Ambiguity

A context-free grammar is **ambiguous** if there are two or more parse trees (or leftmost/rightmost derivations) for some sentence. Otherwise, it is **unambiguous**.

#### Example
The example grammar given above is ambiguous, because two different parse trees can lead to the same sentential form.

#### Note
An ambiguous CFG should not be used to define the syntax of a programming language.
* Context-free grammar ambiguity problem is undecidable in general.
* Rules causing ambiguity can sometimes be modified into unambiguous rules by assuming additional constraints.

Also, there is no transformation from an ambiguous CFG to an unambiguous CFG.

#### Disambiguating rules
Assume the operator precedence $* > +$.

$
\begin{eqnarray}
E &\rightarrow& E + T \mid T \\
T &\rightarrow& T * F \mid F \\
F &\rightarrow& [id] \mid  [const]
\end{eqnarray}
$

#### Example
$[id] * [id] + [id]$

<h3><center><i>Parse Tree Using Unambiguous Grammar</i></center></h3>

<img src="./res/04/4_2.png" width="300px" alt="Parse Tree Using Unambiguous Grammar"/>

### Dangling-else

With the grammar
```
<stmt> → if <expr> then <stmt> else <stmt> | if <expr> then <stmt> | <other-stmt>
```
two parse trees can be derived for the string `if E1 then if E2 then S1 else S2`.

<img src="./res/04/4_3.png" width="400px" alt="Dangling-else issue"/>

#### Disambiguating rules
Assume that else matches with the nearest unmatched preceding then.
```
<if-stmt>        → <matched-stmt> | <unmatched-stmt>
<matched-stmt>   → if <expr> then <matched-stmt> else <matched-stmt> | <other-stmt>
<unmatched-stmt> → if <expr> then <matched-stmt> else <unmatched-stmt>
                 | if <expr> then <stmt>
```

## Left-recursion

This was not discussed in class, but I still think it's interesting/important enough to put here.

Because lookahead symbols are used in top-down parsers, a left-recursive production will cause the parser to get stuck. This is due to the fact that expanding a left-recursive rule keep causing a nonterminal to appear on the left. No terminal symbol will ever be derived as a lookahead.

Consider a nonterminal $A$ with the following productions:

$$A\rightarrow A\alpha \mid \beta$$

Left-recursion can be eliminated by using the following:
$$
\begin{eqnarray}
A &\rightarrow& \beta A' \\
A' &\rightarrow& \alpha A' \mid \varepsilon
\end{eqnarray}
$$

In general, left-recursive productions of the form

$$A\rightarrow A\alpha_1 \mid A\alpha_2 \mid \cdots \mid A\alpha_m \mid \beta_1 \mid \beta_2 \mid \cdot \mid \beta_n$$

can be replaced with the following non-left-recursive productions

$$
\begin{eqnarray}
A &\rightarrow& \beta_1 A' \mid \beta_2 A' \mid \cdots \mid \beta_n A' \\
A' &\rightarrow& \alpha_1 A' \mid \alpha_2 A' \mid \cdots \mid \alpha_m A' \mid \varepsilon
\end{eqnarray}
$$

## Left-factoring

This was also not discussed in class, but I also think it's interesting/important enough to put here.

Left factoring is useful when productions of a grammar share a common prefix. The decision to pick between two productions is essentially deferred by left-factoring.

For a grammar,

$$A\rightarrow \alpha \beta_1 \mid \alpha \beta_2$$

it can be left factored as follows:

$$
\begin{eqnarray}
A &\rightarrow& \alpha A' \\
A' &\rightarrow& \beta_1 \mid \beta_2
\end{eqnarray}
$$

## Deterministic parsing methods:

* Universal parsing algorithms such as Cocke-Younger-Kasami or Earley’s algorithm can parse any context-free language but is impractical in that they run in $O(n^3)$ time.
* Top-down LL and bottom-up LR parsing algorithms can parse large subclasses of context-free languages and run in $O(n)$ time.

## Syntax error detection and recovery:

* Majority of errors are syntactic in nature, e.g., unbalanced parentheses in the arithmetic expression
* Most syntax errors are simple ones that can be detected easily by the parser.
* Again, heuristics must be used to recover from syntax errors and full recovery is not possible nor cost-effective. Panic mode recovery ignores several subsequent tokens, e.g., upto a sentence-ending one such as `;` or `end`, and `continue`s.

## Cocke-Younger-Kasami (CYK) parsing
Given a CFG $G = (N, \Sigma, P, S)$ in **Chomsky Normal Form** and an input string $w$, test if $w \in L(G)$. Let $w = a_1 a_2 \dots a_n$ and let $T$ be an $n \times n$ table such that $T[i, j]$ is the set of all nonterminals generating $a_{i} a_{i+1} \cdots a_{j}$. Then, $w \in L(G)$ if and only if $S \in T[1, n]$. $T$ can be constructed by a _dynamic programming_.

In general, an element $T[i,j]$ is computed in the following way:

| Length   | $a_1$ | $a_2$ | $\cdots$ | $a_i$          | $a_{i+1}$ | $a_{i+2}$ | $\cdots$ | $a_{n-1}$     | $a_n$         | $\vdots$ | $a_{n-1}$ | $a_n$ |
|----------|-------|-------|----------|----------------|-----------|-----------|----------|---------------|---------------|----------|-----------|-------|
| n        |       |       |          |                |           |           |          |               |               |          |           |       |
| $\cdots$ |       |       |          |                |           |           |          |               |               |          |           |       |
| $j$      |       |       |          | $T[i,j]$       |           |           |          |               |               |          |           |       |
| $j-1$    |       |       |          | $\alpha_{j-1}$ | $\beta_1$ |           |          |               |               |          |           |       |
| $j-2$    |       |       |          | $\alpha_{j-2}$ |           | $\beta_2$ |          |               |               |          |           |       |
| $\vdots$ |       |       |          | $\vdots$       |           |           | $\ddots$ |               |               |          |           |       |
| 2        |       |       |          | $\alpha_2$     |           |           |          | $\beta_{j-2}$ |               |          |           |       |
| 1        |       |       |          | $\alpha_1$     |           |           |          |               | $\beta_{j-1}$ |          |           |       |
|          | $a_1$ | $a_2$ | $\cdots$ | $a_i$          | $a_{i+1}$ | $a_{i+2}$ | $\cdots$ | $a_{i+j-2}$   | $a_{i+j-1}$   | $\cdots$ | $a_{n-1}$ | $a_n$ |

In other words, we consider each combination $\alpha_{k} \beta_{k}$, where $k\in\{1,2,\dots,j-1\}$, and see if there is a production in $G$ of the form $\gamma \rightarrow \alpha_{k} \beta_{k}$. If such a production exists, then $\gamma \in T[i,j]$.

#### Note
$T[i,j]$ is the set of productions that can derive a string beginning at position $i$ (1-indexed) of length $j$.

The goal is to check and see if $S \in T[1, n]$.

#### Complexity
$O(n^2)$ table is constructed.
Each box takes $O(n)$ computations. These boxes consider $O(|N|^2)$ nonterminals: $O(|N|)$ nonterminals $\alpha_k$ and $O(|N|)$ nonterminals $\beta_k$.

Altogether, we obtain $O(n^2)\cdot O(n\cdot|N|^2)=O(n^3)$. The $|N|^2$ term is ignored, since it is a constant factor (the grammar is fixed, so the number of nonterminals is fixed).

#### Example
Parse $w=abba$ with the following CFG:

$
\begin{eqnarray}
S &\rightarrow& AB \mid SA \mid BB \\
A &\rightarrow& AA \mid BA \mid a \\
A &\rightarrow& BB \mid b \\
\end{eqnarray}
$

| Length | $a_1$ | $a_2$ | $a_3$ | $a_4$ | $a_5$ |
|--------|-------|-------|-------|-------|-------|
| 5      | S,A   |       |       |       |       |
| 4      | S     | S,A   |       |       |       |
| 3      | S     | B     | S,A   |       |       |
| 2      | S     | S,B   | S,B   | A     |       |
| 1      | A     | B     | B     | B     | A     |
|        | a     | b     | b     | b     | a     |

## Top-down parsing
Given a CFG $G = (N, \Sigma, P, S)$ and an input string $w$, construct a parse tree for $w$ in $G$ top-down, i.e., start with the start symbol $S$ and expand nonterminals in order to generate $w$.

#### Observation

The main difficulty of the top-down parsing lies in the right choice of the right-hand side (RHS) of a rule when there are multiple RHSs.

#### Note

PDA is the recognition device for CFGs (CFG = PDA). Namely, a program structure is defined by a CFG but whether an input structure is valid or not according to the rules of the CFG is tested by a PDA. Our top-down parser will be a PDA simulating a certain type of action of the CFG.

## Pushdown automaton (PDA)

A finite automaton with an additional stack of unbounded size, defined formally as $M = (Q, \Sigma, \Gamma, \delta, q_0, Z_0, F)$, where $Q$ is the state set, $\Sigma$ is the input alphabet, $\Gamma$ is the stack alphabet, $\delta: Q \times (\Sigma \cup \{\varepsilon\}) \times \Gamma \rightarrow 2^{Q \times \Gamma^*}$ is the transition function, $q_0$ is the initial state, $Z_0 \in \Gamma$ is the stack bottom marker, and $F \subseteq Q$ is the set of accepting states.

### Configuration
A triple $(q, x{\uparrow}y, \gamma)$ indicating the PDA’s situation that it is in state $q$ after consuming the prefix $x$ of the input string $xy$, it is scanning the first symbol of $y$ on the input tape, and the stack content is $\gamma$, where the first symbol of $\gamma$ is the stack top symbol.

### Accepting computation
A sequence of configurations $(q_0, {\uparrow}w, Z_0) \vdash \cdots \vdash (q, w{\uparrow}, \gamma)$, where $q \in F$ (the last one in this sequence is an accepting configuration), i.e., a PDA accepts at the end of the input tape if it is in an accepting state. Assume w.l.o.g. that the PDA accepts with the empty stack (so, $\gamma = \varepsilon$).

### Deterministic PDA (DPDA)
A PDA with at most one possible next action at any point of computation

#### Example

DPDA for $L = \{a^n b^n\ |\ n \ge 1\}$

1. Push a’s into the stack.
2. Pop a’s while reading b’s.
3. Accept if the stack is empty at the end of the input tape.

$M = (\{q_0,q_1\}, \{a, b\}, \{a, Z_0\}, \delta, q_0, Z_0, \{q_1\})$, where

$
\begin{align}
\delta(q_0,a,Z_0) = \{(q_0, aZ_0)\} \\
\delta(q_0,a,a) = \{(q_0, aa)\} \\
\delta(q_0,a,a) = \{(q_1, \varepsilon)\} \\
\delta(q_1,b,a) = \{(q_1, \varepsilon)\} \\
\delta(q_1,\varepsilon,Z_0) = \{(q_1, \varepsilon)\} \text{ // empty the stack to accept}
\end{align}
$

#### An accepting computation:

$
\begin{eqnarray}
(q_0, {\uparrow}aabb, Z_0) &\vdash& (q_0, a{\uparrow}abb, aZ_0) \\
&\vdash& (q_0, aa{\uparrow}bb, aaZ_0) \\
&\vdash& (q_1, aab{\uparrow}b, aZ_0) \\
&\vdash& (q_1, aabb{\uparrow}, Z_0) \\
&\vdash& (q_1, aabb{\uparrow}, \varepsilon)
\end{eqnarray}
$

#### Note
DPDA ⊊ PDA, i.e., there is a language defined by a PDA but not by any DPDA.

#### Example
Example. $L = \{ww^R\ |\ w \in \{a, b\}^*\} ∈$ PDA $–$ DPDA.

1. Similar to the above construction.
2. But the mid-point must be guessed.

$M = (\{q_0,q_1\}, \{a, b\}, \{a, Z_0\}, \delta, q_0, Z_0, \{q_1\})$, where

$
\begin{align}
\delta(q_0,X,Z_0) = \{(q_0, XZ_0)\}\ \forall X\in\{a,b\} \\
\delta(q_0,X,X) = \{(q_0, XX), (q_1,\varepsilon)\}\ \forall X\in\{a,b\} \text{ // multiple choices} \\
\delta(q_0,X,Y) = \{(q_0, XY)\}\ \forall X\in\{a,b\} \text{ if } X\ne Y \\
\delta(q_1,X,X) = \{(q_1, \varepsilon)\} \forall X\in\{a,b\} \\
\delta(q_1,\varepsilon,Z_0) = \{(q_1, \varepsilon)\}
\end{align}
$

## CFG to PDA

Given a CFG $G = (N, \Sigma, P, S)$, we construct a PDA M such that $L(M) = L(G)$. For any input $w \in \Sigma^*$, $M$ will simulate a leftmost derivation of $w$ in $G$ and accept $w$ if and only if $w$ can be generated by $G$.

1. Push $S$ into stack. $M$ will check if the stack content $S$ can convert to the unscanned portion $w$ of the input tape by using the rules of $G$.
2. If $M$ has successfully simulated the leftmost derivation of $G$, such as $S \Rightarrow_{lm}^* xA\gamma$, where $x \in \Sigma^*$ and $A \in N$ (thus, $A$ is the leftmost nonterminal in the current sentential form), then the corresponding configuration of $M$ is $(q, x{\uparrow}y, A{\gamma}Z_0)$. $M$ needs to verify that the current stack content $A\gamma$ can convert to the unscanned portion $y$ of the input tape.
3. It is sufficient to understand how $M$ simulates one step action of $G$ that expands a nonterminal to one of its RHSs, say $A \Rightarrow \alpha$. Replace the stack top symbol $A$ by $\alpha$.
  - If a terminal symbol is exposed on the stack top as the result, then consume an identical input symbol and pop it. Repeat this action.
  - If a nonterminal symbol is exposed on the stack top, then we are done with the one-step simulation, so go back to (3).
4. Accept if the stack is empty at the end of the input tape since it is trivial that the stack content $\varepsilon$ can turn to the unscanned portion $\varepsilon$ of the input tape.

Thus,

$M = (\{q_0,q_1\}, \Sigma, N \cup \Sigma \cup \{Z_0\}, \delta, q_0, Z_0, \{q_1\})$, where

$
\begin{align}
\delta(q_0,\varepsilon,Z_0) = \{(q_1, SZ_0)\}\ \\
\delta(q_1,\varepsilon,A) = \{(q_1, \alpha_j) \mid j=1,2,\dots,n\} \text{ if} A\rightarrow\alpha_1 \mid \alpha_2 \mid \cdots \mid \alpha_n \text{ are } A \text{-rules of } G\\
\delta(q_1,a,a) = \{(q_1, \varepsilon)\}\ \forall a\in\Sigma \\
\delta(q_1,\varepsilon,Z_0) = \{(q_1, \varepsilon)\}
\end{align}
$

#### Example

Grammar:
$S\rightarrow aSbS \mid \varepsilon$

PDA:
$M = (\{q_0,q_1\}, \{a,b\}, \{S,a,b,Z_0\}, \delta, q_0, Z_0, \{q_1\})$, where

$
\begin{align}
\delta(q_0,\varepsilon,Z_0) = \{(q_1, SZ_0)\}\ \\
\delta(q_1,\varepsilon,S) = \{(q_1, aSbS), (q_1, \varepsilon)\} \\
\delta(q_1,a,a) = \{(q_1, \varepsilon)\} \\
\delta(q_1,b,b) = \{(q_1, \varepsilon)\} \\
\delta(q_1,\varepsilon,Z_0) = \{(q_1, \varepsilon)\}
\end{align}
$

An accepting computation:

$
\begin{align}
(q_0, {\uparrow}aababb, Z_0) &\vdash& (q_1, {\uparrow}aababb, SZ_0) \text{ // Push S onto stack} \\
&\vdash& (q_1, {\uparrow}aababb, aSbSZ_0) \\
&\vdash& (q_1, a{\uparrow}ababb, SbSZ_0) \\
&\vdash& (q_1, a{\uparrow}ababb, aSbSbSZ_0) \\
&\vdash& (q_1, aa{\uparrow}babb, SbSbSZ_0) \\
&\vdash& (q_1, aa{\uparrow}babb, bSbSZ_0) \\
&\vdash& (q_1, aab{\uparrow}abb, SbSZ_0) \\
&\vdash& (q_1, aab{\uparrow}abb, aSbSbSZ_0) \\
&\vdash& (q_1, aaba{\uparrow}bb, SbSbSZ_0) \\
&\vdash& (q_1, aaba{\uparrow}bb, bSbSZ_0) \\
&\vdash& (q_1, aabab{\uparrow}b, SbSZ_0) \\
&\vdash& (q_1, aabab{\uparrow}b, bSZ_0) \\
&\vdash& (q_1, aababb{\uparrow}, SZ_0) \\
&\vdash& (q_1, aababb{\uparrow}, Z_0) \\
&\vdash& (q_1, aababb{\uparrow}, \varepsilon) \\
\end{align}
$

### Removing nondeterminism

* Use a universal parsing algorithm such as CYK or Earley’s algorithm. This is slow ($O(n^3)$ time).
* Use a backtracking algorithm such as recursive-descent algorithm (a direct deterministic simulation of the above PDA by using the depth-first search). This is slow.
* Use LL grammars for which the above PDA can be made deterministic. This runs in $O(n)$ time.

### Obs. 1
Our PDA’s action depends mainly on the type of symbols on top of the stack:
* Terminal: completely deterministic
* Nonterminal: multiple choices if there are more than one rule for the same stack top nonterminal

### Obs. 2
Our PDA is a totally blind PDA in that the nonterminal-expanding action does not depend at all on the input symbols. Suppose that we have two $A$-rules $A \rightarrow \alpha \mid \beta$ and consider two derivations that start with these two rules:
* $A \Rightarrow \alpha \Rightarrow^* ab\omega$ meaning that $A\rightarrow\alpha$ can generate $ab$ as the first two terminal symbols
* $A \Rightarrow \beta \Rightarrow^* \tau$, where $\tau$ cannot begin with $ab$

**Question:** If the stack top symbol is $A$ and the next two symbols on the input tape is $ab$, then which $A$-rule must be used?

**Answer:** $A \rightarrow \alpha$, not $A \rightarrow \beta$.

This observation suggests that _looking ahead a few symbols_ from the input tape before consuming them can help decide which rule must be used when a nonterminal is on top of the stack.

## LL(k) grammars

A (proper) subclass of CFGs that permit a deterministic, no backtracking and top-down construction of a parse tree in $O(n)$ time by using:
* **L**eft-to-right scan of input symbols,
* **L**eftmost derivations, and
* **k** lookahead symbols.

### Definition

For $k \ge 0$ and $\tau \in (N \cup \Sigma)^*$, define
* $FIRST_k(\tau) = \{x \in \Sigma^* \mid \tau \Rightarrow_{𝑙𝑚}^∗ x\tau' \text{ and } |x| = k \text{ or } \tau \Rightarrow_{𝑙𝑚}^∗ x \text{ and } |x| < k\}$
* $FOLLOW_k(\tau) = \{x \in \Sigma^* \mid S \Rightarrow_{𝑙𝑚}^∗ \lambda \tau \mu \text{ and } x \in FIRST_{k}(\mu)\}$

### Definition

$LOOKAHEAD_k(A \rightarrow \alpha) = FIRST_k(FIRST_k(\alpha)\cdot FOLLOW_k(A))$

### Theorem

A CFG $G$ is LL(k) iff, for every pair of rules $A \rightarrow \mid \beta$ with the same nonterminal $A$ in the left-hand side, $LOOKAHEAD_k(A \rightarrow \alpha) \cap LOOKAHEAD_k(A \rightarrow \beta) = \emptyset$.

### IMPORTANT NOTES
* Always add $\varepsilon$ to the $FOLLOW$ set of $S$
* Right recursion does not add anything to the $FOLLOW$ set
* Use pointers (or some other bookkeeping method) when there is recursion among different $FOLLOW$ sets

#### Example

Construct the LL(k) parse table for the following CFG $G$, with the smallest value of $k$.

$
\begin{align}
S &\rightarrow& Abc \\
S &\rightarrow& aAcb \\
A &\rightarrow& bA \\
A &\rightarrow& c \\
A &\rightarrow& \varepsilon
\end{align}
$

#### Example

$
\begin{align}
S &\rightarrow& aAd \\
A &\rightarrow& BSC \\
B &\rightarrow& b \\
B &\rightarrow& \varepsilon \\
C &\rightarrow& dB \\
C &\rightarrow& d
\end{align}
$

#### Example

$
\begin{align}
S &\rightarrow& aAbe \\
S &\rightarrow& BCf \\
A &\rightarrow& Sd \\
A &\rightarrow& \varepsilon \\
B &\rightarrow& Cd \\
B &\rightarrow& d \\
C &\rightarrow& dfS \\
C &\rightarrow& b
\end{align}
$